* https://github.com/Azure/azureml-examples/blob/055a7cd66710197682a26a5014db173f521b0d62/sdk/endpoints/online/triton/single-model/online-endpoints-triton.ipynb
* https://github.com/Azure/azureml-examples/blob/c2807d2c42d8491b722c1f5c0276d3c0ba2897c2/sdk/endpoints/online/custom-container/online-endpoints-custom-container.ipynb
* https://github.com/Azure/azureml-examples/blob/c2807d2c42d8491b722c1f5c0276d3c0ba2897c2/sdk/endpoints/online/managed/online-endpoints-simple-deployment.ipynb

In [1]:
# Import required libraries
import datetime
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input
from azure.ai.ml.automl import ImageClassificationSearchSpace
from azure.ai.ml.sweep import (
    Choice,
    Uniform,
    BanditPolicy,
)
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    BatchDeployment,
    Model,
    Environment,
    CodeConfiguration,
    OnlineRequestSettings, ProbeSettings
)
from azure.ai.ml import automl

C:\Users\smamgain\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: .\config.json


In [4]:
compute = ml_client.compute.get('nc6-cluster')

In [4]:
model = Model(
    path=f"azureml://datastores/workspaceblobstore/paths/stable-diffusion-v1.ckpt",
    name="stable-diffusion-v1",
    description="",
    tags={},
)
model = ml_client.create_or_update(model)

In [10]:
env = Environment(
    # name="tensoflow-serving",
    image="docker.io/shivammg/stable-diffusion:v1",
    inference_config={
        "liveness_route": {"port": 5000, "path": "/api/health"},
        "readiness_route": {"port": 5000, "path": "/api/health"},
        "scoring_route": {"port": 5000, "path": "/api/txt2img"},
    },
)
# env = ml_client.create_or_update(env)

In [7]:
# endpoint = ManagedOnlineEndpoint(
#     name="stable-diffusion-" + datetime.datetime.now().strftime("%Y%m%d%H%M"),
#     auth_mode="key",
# )
# endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint)
endpoint = list(ml_client.online_endpoints.list())[0]

In [16]:

red_deployment = ManagedOnlineDeployment(
    name="deployment8",
    endpoint_name='stable-diffusion-202209211753',
    model=model,
    environment=env,
    environment_variables={
        #"MODEL_BASE_PATH": "/var/azureml-app/azureml-models/tfserving-mounted/1",
        #"MODEL_NAME": "half_plus_two",
        "MODEL_DIR": "/var/azureml-app/azureml-models/stable-diffusion-v1/1",
    },
    instance_type="Standard_NC6s_v3",
    instance_count=1,
    # compute=compute,
    request_settings=OnlineRequestSettings(request_timeout_ms=60*1000, max_concurrent_requests_per_instance=2, max_queue_wait_ms=1*1000),
    liveness_probe=ProbeSettings(failure_threshold=30, timeout=5, period=10, initial_delay=10),
    readiness_probe=ProbeSettings(failure_threshold=5, timeout=5, period=5, initial_delay=10),
)
red_deployment = ml_client.begin_create_or_update(red_deployment)

Check: endpoint stable-diffusion-202209211753 exists
Creating/updating online deployment deployment8 

.........................................................................................................................................................................

Done (15m 48s)


In [5]:
# dep = ml_client.online_deployments.get('red3', 'stable-diffusion-202209172147')
# ml_client.environments.get('CliV2AnonymousEnvironment', )

In [20]:
ml_client.online_deployments.delete('red3', 'stable-diffusion-202209172147')

In [9]:
from azure.ai.ml.entities import ProbeSettings
ProbeSettings?

In [30]:
model = Model(name="tfserving-mounted", version="1", path="half_plus_two")

env = Environment(
    image="docker.io/tensorflow/serving:latest",
    inference_config={
        "liveness_route": {"port": 8501, "path": "/v1/models/half_plus_two"},
        "readiness_route": {"port": 8501, "path": "/v1/models/half_plus_two"},
        "scoring_route": {"port": 8501, "path": "/v1/models/half_plus_two:predict"},
    },
)

blue_deployment = ManagedOnlineDeployment(
    name="blue2",
    endpoint_name=endpoint.name,
    model=model,
    environment=env,
    environment_variables={
        "MODEL_BASE_PATH": "/var/azureml-app/azureml-models/tfserving-mounted/1",
        "MODEL_NAME": "half_plus_two",
    },
    instance_type="Standard_F2s_v2",
    instance_count=1,
)
blue_deployment = ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint stable-diffusion-202209172147 exists
Uploading half_plus_two (0.02 MBs): 100%|#####################################| 23847/23847 [00:01<00:00, 23705.96it/s]


Creating/updating online deployment blue2 

...............................................................................................................................................................................

Done (16m 24s)


In [8]:
key = ml_client.online_endpoints.list_keys(endpoint.name)
primary_key = key.as_dict()['primary_key']
with open('online-endpoint-key.txt', 'w') as f:
    f.write(primary_key)

In [8]:
ManagedOnlineDeployment?

In [39]:
OnlineRequestSettings?